In [41]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

In [4]:
standard_format = pd.read_csv('/Users/mayankmahawar/Documents/GitHub/Text Mining/StandardFormat-Output.csv')

In [6]:
print(standard_format.shape)
standard_format.head()

(16, 8)


,Date,Transaction Description,Debit,Credit,Currency,CardName,Transaction,Location
0,12-01-2018,STIC TRAVELS PVT LTD DELHI,0.0,32256,INR,Rahul,Domestic,delhi
1,13-01-2018,SRILANKANUPGRADE KATUNAYAKE,6.0,0,EUR,Rahul,International,katunayake
2,13-01-2018,FLIPKART INTERNET PRIVATE BANGALORE,211687.0,0,INR,Rahul,Domestic,bangalore
3,14-01-2018,HEALTHGUARD LIMITED KATUNAYAKE,0.0,5,USD,Rahul,International,katunayake
4,14-01-2018,FOOT RUB BEDRLIN,20.0,0,POUND,Rahul,International,bedrlin


### File imports

In [491]:
hdfc = pd.read_csv('/Users/mayankmahawar/Documents/GitHub/Text Mining/HDFC-Input-Case1.csv', header=None)
icici = pd.read_csv('/Users/mayankmahawar/Documents/GitHub/Text Mining/ICICI-Input-Case2.csv', header=None)
axis = pd.read_csv('/Users/mayankmahawar/Documents/GitHub/Text Mining/Axis-Input-Case3.csv', header=None)
idfc = pd.read_csv('/Users/mayankmahawar/Documents/GitHub/Text Mining/IDFC-Input-Case4.csv', header=None)

In [492]:
print(hdfc.shape, icici.shape, axis.shape, idfc.shape)

(46, 3) (68, 5) (46, 4) (51, 6)


In [493]:
#Remove rows with all nan values
for i in [hdfc, icici, axis, idfc]:
    i.dropna(inplace=True, how='all')
for i in [hdfc, icici, axis, idfc]:
    i.reset_index(inplace=True, drop=True)

In [494]:
print(hdfc.shape, icici.shape, axis.shape, idfc.shape)

(23, 3) (30, 5) (28, 4) (33, 6)


### Rules

In [613]:
def amount_type_formatting(df, columns):
    column_order = ['Date', 'Transaction', 'Amount']
    df.columns = columns
    hdfc = df[column_order].copy()
    
    date = pd.to_datetime(hdfc.iloc[:,0], errors='coerce')

    m1 = (date.isna()) & ~(hdfc.isin(['International Transactions','Domestic Transactions'])).any(axis=1)
    print(m1)
    card = pd.Series(np.where(m1, hdfc.iloc[:,1], np.nan)).fillna(method='ffill')
    print(card)

    m2 = (hdfc.isin(['International Transactions','Domestic Transactions'])).any(axis=1)
    transactions = pd.Series(np.where(m2, hdfc.iloc[:,1], np.nan)).fillna(method='ffill')
    transactions = pd.Series(np.where(~np.isnat(date), transactions, np.nan))
    
    #debit_credit columns availaibility check
    m3 = (hdfc.isin(['Debit','Credit'])).any(axis=1)
    #debit_credit columns availaibility check
    m4 = (hdfc.isin(['Amount'])).any(axis=1)
    
    m5 = hdfc.iloc[:, 2].str.contains('cr') | hdfc.iloc[:, 2].str.contains('Cr')
    credit = pd.Series(np.where(m5, hdfc.iloc[:,2], np.nan)).str.replace(' cr| Cr', '', regex=True).astype('float')

    m6 = ((hdfc.iloc[:, 2].str.contains(' cr$| Cr$|Amount', regex=True)) == False)
    debit = pd.Series(np.where(m6, hdfc.iloc[:,2], np.nan)).astype('float')
    
    m7 = (transactions=='International Transactions')  & ~np.isnat(date)
    currency = pd.Series(np.where(m7, pd.Series([i[-1] if isinstance(i, list) else '' for i in hdfc.iloc[:,1].str.split()]), np.nan))
    m7_1 = (transactions=='Domestic Transactions') & ~np.isnat(date)
    currency = currency.fillna(pd.Series(np.where(m7_1, 'INR', np.nan)))
    
    m8 = (transactions=='Domestic Transactions')
    location = pd.Series(np.where(m8, pd.Series([i[-1] if isinstance(i, list) else '' for i in hdfc.iloc[:,1].str.split()]), np.nan))
    m8_1 = (transactions=='International Transactions')
    return (hdfc.iloc[:,1].str.split())
    location = location.fillna(pd.Series(np.where(m8_1, [i[-2] if (len(i)>2)&(isinstance(i, list)) else '' for i in hdfc.iloc[:,1].str.split()], np.nan)))
    
    m9 = (transactions=='International Transactions') & ~np.isnat(date)
    mod_transaction = pd.Series(np.where(m9, pd.Series([i[:-1] if isinstance(i, list) else '' for i in hdfc.iloc[:,1].str.split()]), np.nan))
    mod_transaction = mod_transaction.str.join(' ')
    m9_1 = (transactions=='Domestic Transactions')
    mod_transaction = mod_transaction.fillna(pd.Series(np.where(m9_1, hdfc.iloc[:,1], np.nan)))

    df = {
        'Date' : date,
        'Transaction Description' : mod_transaction,
        'Debit' : debit,
        'Credit' : credit,
        'Currency' : currency,
        'Card Name' : card,
        'Transaction' : transactions,
        'Location' : location
    }
    df = pd.DataFrame(df)
    df = df.loc[df['Date'].notnull()].reset_index(drop=True)
    
    return df

In [564]:
amount_type_formatting(hdfc, columns = ['Date', 'Transaction', 'Amount'])

,Date,Transaction Description,Debit,Credit,Currency,Card Name,Transaction,Location
0,2018-12-01,STIC TRAVELS PVT LTD DELHI,NaN,32256.0,INR,Rahul,Domestic Transactions,DELHI
1,2018-01-13,FLIPKART INTERNET PRIVATE BANGALORE,211687.00,NaN,INR,Rahul,Domestic Transactions,BANGALORE
2,2018-01-14,FLEMINGO DUTY FREE Mumbai,18796.99,NaN,INR,Rahul,Domestic Transactions,Mumbai
3,2018-01-14,AIRTEL PAYMENT MUMBAI,1297.00,NaN,INR,Rahul,Domestic Transactions,MUMBAI
4,2018-01-14,AIRTEL PAYMENT CHENNAI,NaN,902.0,INR,Rahul,Domestic Transactions,CHENNAI
5,2018-01-22,FLIPKART INTERNET PRIVATE BANGALORE,5000.00,NaN,INR,Rahul,Domestic Transactions,BANGALORE
6,2018-01-17,PAYTM NOIDA,165.00,NaN,INR,Ritu,Domestic Transactions,NOIDA
7,2018-01-17,BIKANERVALA GURGAON,325.00,NaN,INR,Ritu,Domestic Transactions,GURGAON
8,2018-01-19,VFS GLOBAL SERVICES PVT MUMBAI,1200.00,NaN,INR,Ritu,Domestic Transactions,MUMBAI
9,2018-01-21,Amazon Seller Services BANGALORE,NaN,2155.0,INR,Ritu,Domestic Transactions,BANGALORE


In [495]:
idfc.iloc[:,1] = idfc.iloc[:,1].fillna(idfc.iloc[:,3]).fillna(idfc.iloc[:,4])
idfc = idfc.iloc[:,0:3]
idfc = idfc.dropna(how='all').reset_index(drop=True)

In [614]:
x = amount_type_formatting(idfc, columns = ['Transaction', 'Date', 'Amount'])

0     False
1      True
2      True
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13     True
14     True
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25     True
26     True
27    False
28    False
29    False
30    False
31    False
32    False
dtype: bool
0                                         NaN
1          Transaction Details               
2          Transaction Details               
3          Transaction Details               
4          Transaction Details               
5          Transaction Details               
6          Transaction Details               
7          Transaction Details               
8          Transaction Details               
9          Transaction Details               
10         Transaction Details               
11         Transaction Details               
12         Transaction Details          

/Users/mayankmahawar/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [608]:
idfc

,Transaction,Date,Amount
0,NaN,Domestic Transactions,NaN
1,Transaction Details,Date,Amount
2,NaN,Rahul,NaN
3,JUNOON RESTRO GURGAON,13-12-2017,1255
4,POONAM SERVICE STATION GURGAON,16-12-2017,17 Cr
5,KABI BESPOKE BOUTIQUE GURGAON,16-12-2017,220
6,CLEARTRIP TRAVEL SERVIC MUMBAI,16-12-2017,14849
7,INDIAN RAILWAY CATERINGNEW DELHI,21-12-2017,1684
8,CLEARTRIP TRAVEL SERVIC MUMBAI,22-12-2017,5205
9,MSW*BLISS HOSPITALITIE Delhi NCR,22-12-2017,3764
